In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import openai
import pandas as pd

openai.api_key = "api-key" ##your api code

##Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('directory file', sheet_name='Sheet1') ## your file
print(df.shape)

(6, 8)


In [ ]:
print(df)

##Starts processing

In [ ]:
def analyze_sentiment_with_gpt3(text):
    prompt = f"Analyze the following restaurant review and extract each aspect (food, service, price, ambience, place), sentiment (positive, negative, or null), and opinions. If the review contains no facets, the corresponding cell is filled with \"null\". result format, aspect;sentiment;opinion\n{text}"

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt,
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    result_text = response.choices[0].text.strip()
    
    result_dict = {"aspect": [], "sentiment": [], "opinion": []}
    for line in result_text.strip().split('\n'):
        line_parts = line.split(';')
        if len(line_parts) == 3:
            aspect, sentiment, opinion = line_parts
            result_dict["aspect"].append(aspect)
            result_dict["sentiment"].append(sentiment)
            result_dict["opinion"].append(opinion)

    return result_dict

In [ ]:
# Apply the analyze_sentiment_with_gpt3 function to the 'review' column and expand the result into new columns
result_columns = df['review_en'].apply(analyze_sentiment_with_gpt3).apply(pd.Series)

# Rename the columns in the result_columns DataFrame
result_columns.columns = ['aspect', 'sentiment', 'opinion']

# Concatenate the result_columns DataFrame with the original subset_df DataFrame
new_df = pd.concat([df, result_columns], axis=1)

In [ ]:
new_df

In [ ]:
#save in your google drive
new_df.to_excel('filename.xlsx')
!cp filename.xlsx "drive/My Drive/Data scraping/Sundanese Results"